In [ ]:
from google.colab import files

# Prompt to upload a file
uploaded = files.upload()

Saving test_test.csv to test_test.csv


In [ ]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

sns.set(style='dark', context='notebook', palette='deep')
pd.options.display.max_columns = 100

In [ ]:
# Step 1: Read the Excel file into a DataFrame
test_test = 'test_test.csv'
df = pd.read_csv(test_test)

In [ ]:
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
0,655269,53478,63558,86.54,67,22807,45,1497,1/1/1974,Salaried,3/11/2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
1,723482,55513,63163,89.45,67,22807,45,1497,20-05-85,Self employed,20-11-18,6,1998,1,1,0,0,0,0,749,C-Very Low Risk,2,1,0,43898,48780,48780,0,0,0,0,0,0,5605,0,1,0,0yrs 8mon,1yrs 0mon,1
2,758529,65282,84320,79.93,78,23135,86,2071,14-10-95,Salaried,29-11-18,4,1646,1,1,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
3,763449,46905,63896,76.58,78,17014,45,2070,1/6/1973,Self employed,29-11-18,4,1646,1,1,0,0,0,0,14,Not Scored: Only a Guarantor,1,1,1,132480,255000,255000,0,0,0,0,0,0,0,0,0,0,2yrs 5mon,2yrs 5mon,0
4,708663,51428,63896,86.08,78,17014,45,2069,1/6/1972,Salaried,17-11-18,4,1646,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0


## Let me try and define my target varriable

In [ ]:

# Assuming your dataset is named 'df'
# You'll need to adjust column names and criteria based on your data

# Segment by Employment Type and Credit History Length
salaried_no_history = df[(df['Employment.Type'] == 'Salaried') & (df['CREDIT.HISTORY.LENGTH'] == '0yrs 0mon')]
self_employed_high_risk = df[(df['Employment.Type'] == 'Self employed') & (df['PERFORM_CNS.SCORE.DESCRIPTION'] == 'High Risk')]

# Define target variables for each segment based on Number of Accounts
salaried_no_history['loan_default'] = (salaried_no_history['PRI.NO.OF.ACCTS'] > 5).astype(int)
self_employed_high_risk['loan_default'] = (self_employed_high_risk['PRI.NO.OF.ACCTS'] > 3).astype(int)

# Concatenate segmented data back together
segmented_data = pd.concat([salaried_no_history, self_employed_high_risk])

# Check the distribution of the newly created target variable within each segment
print("Salaried Customers with No Credit History - Loan Default:")
print(segmented_data[(segmented_data['Employment.Type'] == 'Salaried') & (segmented_data['CREDIT.HISTORY.LENGTH'] == '0yrs 0mon')]['loan_default'].value_counts())

print("\nSelf-Employed High-Risk Customers - Loan Default:")
print(segmented_data[(segmented_data['Employment.Type'] == 'Self employed') & (segmented_data['PERFORM_CNS.SCORE.DESCRIPTION'] == 'High Risk')]['loan_default'].value_counts())


Salaried Customers with No Credit History - Loan Default:
0    24502
Name: loan_default, dtype: int64

Self-Employed High-Risk Customers - Loan Default:
Series([], Name: loan_default, dtype: int64)


In [ ]:
segmented_data.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,655269,53478,63558,86.54,67,22807,45,1497,1/1/1974,Salaried,3/11/2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
2,758529,65282,84320,79.93,78,23135,86,2071,14-10-95,Salaried,29-11-18,4,1646,1,1,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
4,708663,51428,63896,86.08,78,17014,45,2069,1/6/1972,Salaried,17-11-18,4,1646,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
7,714975,48743,63896,81.38,78,17014,45,2079,24-11-95,Salaried,19-11-18,4,1646,1,1,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
8,712795,50295,63896,82.35,78,17014,45,2071,1/1/1988,Salaried,18-11-18,4,1646,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0


In [ ]:
# Assuming 'df' is the name of your DataFrame
unique_loan_default_values = segmented_data['loan_default'].unique()
print("Unique values in the 'loan_default' column:")
print(unique_loan_default_values)


Unique values in the 'loan_default' column:
[0]


In [ ]:

# Assuming your dataset is named 'df'
# You'll need to adjust column names and criteria based on your data

# Segment by Employment Type and Credit History Length
salaried_no_history = df[(df['Employment.Type'] == 'Salaried') & (df['CREDIT.HISTORY.LENGTH'] == '0yrs 0mon')]
self_employed_high_risk = df[(df['Employment.Type'] == 'Self employed') & (df['PERFORM_CNS.SCORE.DESCRIPTION'] == 'High Risk')]

# Define target variables for each segment based on Number of Accounts
salaried_no_history['loan_default'] = (salaried_no_history['PRI.NO.OF.ACCTS'] > 5).astype(int)
self_employed_high_risk['loan_default'] = (self_employed_high_risk['PRI.NO.OF.ACCTS'] > 3).astype(int)

# Concatenate segmented data back together
segmented_data = pd.concat([salaried_no_history, self_employed_high_risk])

# Check the distribution of the newly created target variable within each segment
print("Salaried Customers with No Credit History - Loan Default:")
print(segmented_data[(segmented_data['Employment.Type'] == 'Salaried') & (segmented_data['CREDIT.HISTORY.LENGTH'] == '0yrs 0mon')]['loan_default'].value_counts())

print("\nSelf-Employed High-Risk Customers - Loan Default:")
print(segmented_data[(segmented_data['Employment.Type'] == 'Self employed') & (segmented_data['PERFORM_CNS.SCORE.DESCRIPTION'] == 'High Risk')]['loan_default'].value_counts())
